# scrape_charging_stations.ipynb
**This notebook downloads the record of public EV charging stations from [NREL's Alternative Fuel Data Center](https://developer.nrel.gov/docs/transportation/alt-fuel-stations-v1/) and stores the outputs in the corresponding folder `data/evse/raw/`**
+ Each downloaded file is recorded with today's date in its filename, e.g. `EV_charging_stations_WA (as of 2022-02-27).csv`
+ This notebook also updates the most recent file for each of the datasets using the ``update_latest_file()`` function from the `utils.py` library, and stores it in a separate file, e.g. `EV_charging_stations_WA.csv` (without any date note)
+ Time required for scraping: ~1 second

In [1]:
import pandas as pd
import datetime as dt
import censusgeocode as cg
from uszipcode import SearchEngine
search = SearchEngine()

from utils import scrape_from_api,update_latest_file
from keys import AFDC_API_KEY
from config.GLOBAL import *

C:\Users\steff\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
folder = "data/evse/"
fname_base = "EV_charging_stations_WA"
today = dt.date.today().strftime("%Y-%m-%d")
filename = folder + fname_base + ".csv"
filename_raw = folder + "raw/" + fname_base + " (as of {0:s}).csv".format(today)

url = "https://developer.nrel.gov/api/alt-fuel-stations/v1.csv"
params = {}
params["api_key"] = AFDC_API_KEY
params["fuel_type"] = "ELEC"
params["state"] = "WA"

In [46]:
scrape_from_api(url, params, filename_raw)

In [47]:
update_latest_file(fname_base, folder)

latest file: 'EV_charging_stations_WA (as of 2022-11-08).csv'


In [52]:
#rename column headers in latest file
df = pd.read_csv(filename)
df_key = pd.read_csv("config/EV_charging_stations_key.csv")
df = df.rename(columns={row[0]: row[1] for row in df_key[["full name", "name"]].values})
df.to_csv(filename, index=False)

In [53]:
df.columns

Index(['fuel_type_code', 'station_name', 'street_address',
       'intersection_directions', 'city', 'state', 'zip', 'plus4',
       'station_phone', 'status_code', 'expected_date',
       'groups_with_access_code', 'access_days_time', 'cards_accepted',
       'bd_blends', 'ng_fill_type_code', 'ng_psi', 'l1_count', 'l2_count',
       'dcfc_count', 'ev_other_info', 'ev_network', 'ev_network_web',
       'geocode_status', 'latitude', 'longitude', 'date_last_confirmed', 'ID',
       'updated_at', 'owner_type_code', 'federal_agency_ID',
       'federal_agency_name', 'open_date', 'hydrogen_status_link',
       'ng_vehicle_class', 'LPG_primary', 'e_blender_pump',
       'ev_connector_types', 'country', 'intersection_directions_french',
       'access_days_time_french)', 'bd_blends_french)',
       'groups_with_access_code_french', 'hydrogen_is_retail', 'access_code',
       'access_detail_code', 'federal_agency_code', 'facility_type',
       'cng_dispenser_num', 'cng_on_site_renewable_source

In [56]:
#add 2020 census tract field to data
df = pd.read_csv(filename)

df["census_tract_2020"] = None
for ind,row in df.iterrows():
    if ind % 10 == 0:
        print(ind, "", end="")
    res = cg.coordinates(df.loc[ind, "longitude"], df.loc[ind, "latitude"])#, vintage="Census2010_Current")
    df.loc[ind, "census_tract_2020"] = res["Census Tracts"][0]["GEOID"]
df["census_tract_2020"] = df["census_tract_2020"].astype("int64")

0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450 460 470 480 490 500 510 520 530 540 550 560 570 580 590 600 610 620 630 640 650 660 670 680 690 700 710 720 730 740 750 760 770 780 790 800 810 820 830 840 850 860 870 880 890 900 910 920 930 940 950 960 970 980 990 1000 1010 1020 1030 1040 1050 1060 1070 1080 1090 1100 1110 1120 1130 1140 1150 1160 1170 1180 1190 1200 1210 1220 1230 1240 1250 1260 1270 1280 1290 1300 1310 1320 1330 1340 1350 1360 1370 1380 1390 1400 1410 1420 1430 1440 1450 1460 1470 1480 1490 1500 1510 1520 1530 1540 1550 1560 1570 1580 1590 1600 1610 1620 1630 1640 1650 1660 1670 1680 1690 1700 1710 1720 1730 1740 1750 1760 1770 1780 1790 1800 1810 1820 1830 1840 1850 1860 1870 1880 1890 1900 

In [57]:
df

,fuel_type_code,station_name,street_address,intersection_directions,city,state,zip,plus4,station_phone,status_code,...,lpg_nozzle_types,hydrogen_pressures,hydrogen_standards,cng_fill_type_code,cng_psi,cng_vehicle_class,lng_vehicle_class,ev_on_site_renewable_source,restricted_access,census_tract_2020
0,ELEC,City of Lacey - City Hall Parking,420 College St,"At 3rd Ave SE, next to police station",Lacey,WA,98503,NaN,360-491-3214,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,53067011200
1,ELEC,Seattle-Tacoma International Airport - General...,17801 Pacific Hwy S,"5th floor parking garage; rows D, G, and I",Seattle,WA,98188,NaN,206-787-5388,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,53033028402
2,ELEC,Haggen Food & Pharmacy,1313 Cooper Point Rd SW,In front of the building on the left hand side,Olympia,WA,98502,NaN,360-754-1428,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,53067010510
3,ELEC,Avista Corp,1411 E Mission Ave,NaN,Spokane,WA,99252,NaN,509-489-0500,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,53063001800
4,ELEC,Steam Plant Grill,159 S Lincoln St,NaN,Spokane,WA,99201,NaN,509-777-3900,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,53063003500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1902,ELEC,University of Washington - Parking Area W40,814 NE Northlake Pl,NaN,Seattle,WA,98105,NaN,206-221-3701,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,53033005202
1903,ELEC,University of Washington - Parking Lot N22,4060 E Stevens Way NE,NaN,Seattle,WA,98195,NaN,206-221-3701,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,53033005303
1904,ELEC,Union Bay Plaza,4516 Union Bay Pl NE,NaN,Seattle,WA,98105,NaN,NaN,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,53033004202
1905,ELEC,OLY CITY HALL OLY CITY HALL 1,601 4th Ave E,NaN,Olympia,WA,98501,NaN,888-758-4389,T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53067010100


In [58]:
#add 2010 census tract
df["census_tract_2010"] = 0
for ind,row in df.iterrows():
    if df.loc[ind, "census_tract_2020"] in df_tract_20_10:
        df.loc[ind, "census_tract_2010"] = df_tract_20_10.loc[df.loc[ind, "census_tract_2020"]]
# df["census_tract_2010"] = df["census_tract_2010"].astype(int)
df

,fuel_type_code,station_name,street_address,intersection_directions,city,state,zip,plus4,station_phone,status_code,...,hydrogen_pressures,hydrogen_standards,cng_fill_type_code,cng_psi,cng_vehicle_class,lng_vehicle_class,ev_on_site_renewable_source,restricted_access,census_tract_2020,census_tract_2010
0,ELEC,City of Lacey - City Hall Parking,420 College St,"At 3rd Ave SE, next to police station",Lacey,WA,98503,NaN,360-491-3214,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,53067011200,53067011200
1,ELEC,Seattle-Tacoma International Airport - General...,17801 Pacific Hwy S,"5th floor parking garage; rows D, G, and I",Seattle,WA,98188,NaN,206-787-5388,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,53033028402,53033028402
2,ELEC,Haggen Food & Pharmacy,1313 Cooper Point Rd SW,In front of the building on the left hand side,Olympia,WA,98502,NaN,360-754-1428,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,53067010510,53067010510
3,ELEC,Avista Corp,1411 E Mission Ave,NaN,Spokane,WA,99252,NaN,509-489-0500,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,53063001800,53063001800
4,ELEC,Steam Plant Grill,159 S Lincoln St,NaN,Spokane,WA,99201,NaN,509-777-3900,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,53063003500,53063003500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1902,ELEC,University of Washington - Parking Area W40,814 NE Northlake Pl,NaN,Seattle,WA,98105,NaN,206-221-3701,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,53033005202,53033005200
1903,ELEC,University of Washington - Parking Lot N22,4060 E Stevens Way NE,NaN,Seattle,WA,98195,NaN,206-221-3701,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,53033005303,53033005302
1904,ELEC,Union Bay Plaza,4516 Union Bay Pl NE,NaN,Seattle,WA,98105,NaN,NaN,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,53033004202,53033004200
1905,ELEC,OLY CITY HALL OLY CITY HALL 1,601 4th Ave E,NaN,Olympia,WA,98501,NaN,888-758-4389,T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53067010100,53067010100


In [42]:
print(len(df["census_tract_2020"].unique()))
print(len(df["census_tract_2010"].unique()))

595
548


In [60]:
df.to_csv(filename.split(".")[0] + "_2010tracts.csv")

In [61]:
multiindex = pd.MultiIndex.from_product([times, census_tracts_2020], names=["time", "census_tract_2020"])
# columns = ["n_ev", "n_bev", "n_phev", "m_ev", "m_bev", "m_phev"]

In [62]:
# create final dataframe and add county and ZIP code column to it
final_df = pd.DataFrame(index=multiindex, columns=["county", "zip_code"])
# for zip_code in zip_codes:
#     final_df.loc[(slice(None), zip_code), "county"] = search.by_zipcode(zip_code).county.replace(" County", "")
for census_tract in census_tracts_2020:
    if census_tract in tract_to_zip.index:
        zip_code = tract_to_zip.loc[census_tract, "zip_code"]
        final_df.loc[(slice(None), census_tract), "zip_code"] = zip_code
        final_df.loc[(slice(None), census_tract), "county"] = search.by_zipcode(zip_code).county.replace(" County", "")
final_df

county zip_code
time       census_tract_2020                 
2011-01-31 53001950100         Adams    99169
           53001950200         Adams    99371
           53001950301           NaN      NaN
           53001950302           NaN      NaN
           53001950303           NaN      NaN
...                              ...      ...
2022-09-30 53077940003        Yakima    98903
           53077940005        Yakima    98948
           53077940006        Yakima    98948
           53077940007           NaN      NaN
           53077940008           NaN      NaN

[251544 rows x 2 columns]

In [63]:
#calculate predictor variable
#number of installed L2 and DCFC station locations by zip code and month
# df_evse = pd.read_csv("data/evse/EV_charging_stations_WA.csv")
df = pd.read_csv("data/evse/EV_charging_stations_WA_2010tracts.csv")

# df_evse = df_evse.drop("Unnamed: 0", axis=1)
# print(df_evse.columns)
# df_evse.columns = df_evse_key["name"].to_list() + ["census_tract_2020"]
# df_evse.columns = df_evse_key["name"].to_list() + ["census_tract_2020", "census_tract_2010"]
# print(df_evse.columns)
# k

# print(df_evse["open_date"].min())
df = df[df["open_date"] != "0022-07-22"]

df["open_date"] = pd.to_datetime(df["open_date"]).dt.date
# j
df["l2_count"] = df["l2_count"].fillna(0).astype(int)
df["dcfc_count"] = df["dcfc_count"].fillna(0).astype(int)

# df_evse["has_l2_or_dcfc"] = df_evse["l2_count"] > 0 #boolean if location has at least one L2 charger
df["has_l2_or_dcfc"] = (df["l2_count"] > 0) | (df["dcfc_count"] > 0) #boolean if location has at least one L2 or DCFC charger

final_df["n_evse"] = 0
for time in times:
    print(time, "", end="")
    # for zip_code in zip_codes:
    for census_tract in census_tracts_2020:
        # dff = df_evse[(df_evse["open_date"] < time) & (df_evse["zip"] == zip_code)] #create helper dataframe containing all stations built in this zip code until this time
        dff = df[(df["open_date"] < time) & (df["census_tract_2020"] == census_tract)] #create helper dataframe containing all stations built in this census tract until this time
        n_evse = sum(dff["has_l2_or_dcfc"]) #counts number of locations with at least one charging station in that ZIP code and for that month
        final_df.loc[(time, census_tract), "n_evse"] = n_evse

print("done")

2011-01-31 2011-02-28 2011-03-31 2011-04-30 2011-05-31 2011-06-30 2011-07-31 2011-08-31 2011-09-30 2011-10-31 2011-11-30 2011-12-31 2012-01-31 2012-02-29 2012-03-31 2012-04-30 2012-05-31 2012-06-30 2012-07-31 2012-08-31 2012-09-30 2012-10-31 2012-11-30 2012-12-31 2013-01-31 2013-02-28 2013-03-31 2013-04-30 2013-05-31 2013-06-30 2013-07-31 2013-08-31 2013-09-30 2013-10-31 2013-11-30 2013-12-31 2014-01-31 2014-02-28 2014-03-31 2014-04-30 2014-05-31 2014-06-30 2014-07-31 2014-08-31 2014-09-30 2014-10-31 2014-11-30 2014-12-31 2015-01-31 2015-02-28 2015-03-31 2015-04-30 2015-05-31 2015-06-30 2015-07-31 2015-08-31 2015-09-30 2015-10-31 2015-11-30 2015-12-31 2016-01-31 2016-02-29 2016-03-31 2016-04-30 2016-05-31 2016-06-30 2016-07-31 2016-08-31 2016-09-30 2016-10-31 2016-11-30 2016-12-31 2017-01-31 2017-02-28 2017-03-31 2017-04-30 2017-05-31 2017-06-30 2017-07-31 2017-08-31 2017-09-30 2017-10-31 2017-11-30 2017-12-31 2018-01-31 2018-02-28 2018-03-31 2018-04-30 2018-05-31 2018-06-30 2018-07-31

In [64]:
final_df

county zip_code  n_evse
time       census_tract_2020                         
2011-01-31 53001950100         Adams    99169       0
           53001950200         Adams    99371       0
           53001950301           NaN      NaN       0
           53001950302           NaN      NaN       0
           53001950303           NaN      NaN       0
...                              ...      ...     ...
2022-09-30 53077940003        Yakima    98903       0
           53077940005        Yakima    98948       0
           53077940006        Yakima    98948       0
           53077940007           NaN      NaN       0
           53077940008           NaN      NaN       2

[251544 rows x 3 columns]

In [66]:
final_df["n_evse"].sum()

65652

In [65]:
final_df.to_csv("data/evse/evse_2.csv")

In [68]:
#express in terms of 2010 census tracts as the index
final_df_2010tracts = final_df.reset_index()
# final_df_2010tracts["n_evse"] = final_df_2010tracts["n_evse"].fillna(0)
final_df_2010tracts["census_tract_2010"] = final_df_2010tracts["census_tract_2020"].map(df_tract_20_10.to_dict())
# final_df_2010tracts[["census_tract_2020", "zip_code", "county"]] = final_df_2010tracts[["census_tract_2020", "zip_code", "county"]].astype(str)

final_df_2010tracts2 = final_df_2010tracts[["time", "census_tract_2010", "census_tract_2020", "zip_code", "county", "n_evse"]].groupby(["time", "census_tract_2010"]).sum() #for count variables (n_evse)
# final_df_2010tracts3 = final_df_2010tracts[["time", "census_tract_2010", "census_tract_2020", "zip_code"] + columns[3:]].groupby(["time", "census_tract_2010"]).mean() #product variety (m_ev, m_bev, m_phev)
# final_df_2010tracts4 = final_df_2010tracts2.merge(final_df_2010tracts3, on=["time", "census_tract_2010"])
# final_df_2010tracts4 = final_df_2010tracts4.drop(["census_tract_2020_y", "zip_code_y"], axis=1)
final_df_2010tracts2 = final_df_2010tracts2.drop("census_tract_2020", axis=1)
# final_df_2010tracts4 = final_df_2010tracts4.rename({"census_tract_2020_x": "census_tract_2020", "zip_code_x": "zip_code"}, axis=1)

final_df_2010tracts2.to_csv("data/evse/evse_2010tracts.csv")
# final_df_2010tracts2.to_csv("data/evse/evse_2010tracts_2.csv")
final_df_2010tracts2

n_evse
time       census_tract_2010        
2011-01-31 53001950100             0
           53001950200             0
           53001950300             0
           53001950400             0
           53001950500             0
...                              ...
2022-09-30 53077940002             0
           53077940003             0
           53077940004             2
           53077940005             0
           53077940006             0

[205437 rows x 1 columns]

In [ ]:
#express in terms of 2010 census tracts as the index (WRONG METHOD, DO NOT USE)
final_df_2010tracts = final_df.reset_index()
final_df_2010tracts["census_tract_2010"] = final_df_2010tracts["census_tract_2020"].map(df_tract_20_10.to_dict())
final_df_2010tracts = final_df_2010tracts.set_index(["time", "census_tract_2010"])
final_df_2010tracts.to_csv("data/evse/evse_2010tracts.csv")
final_df_2010tracts